# Validate data in the staging tables using SQL


In [1]:
import csv

import json

import math
import numpy as np
import pandas as pd

import psycopg2

In [2]:
#
# function to run a select query and return rows in a pandas dataframe
# pandas puts all numeric values from postgres to float
# if it will fit in an integer, change it to integer
#

def my_select_query_pandas(query, rollback_before_flag, rollback_after_flag):
    "function to run a select query and return rows in a pandas dataframe"
    
    if rollback_before_flag:
        connection.rollback()
    
    df = pd.read_sql_query(query, connection)
    
    if rollback_after_flag:
        connection.rollback()
    
    # fix the float columns that really should be integers
    
    for column in df:
    
        if df[column].dtype == "float64":

            fraction_flag = False

            for value in df[column].values:
                
                if not np.isnan(value):
                    if value - math.floor(value) != 0:
                        fraction_flag = True

            if not fraction_flag:
                df[column] = df[column].astype('Int64')
    
    return(df)
    

In [3]:
connection = psycopg2.connect(
    user = "postgres",
    host = "postgres",
    port = "5432",
    database = "postgres"
)

In [4]:
cursor = connection.cursor()

# Validate the data types in the staging table stage_1_peak_sales

* sale_id - validate that is is numeric
* sales_date - validate that it is a date
* sub_total - validate that it is numeric
* tax - validate that it is numeric
* total_amount - validate that it is numeric

In [5]:
rollback_before_flag = True
rollback_after_flag = True

query = """

SELECT 
stage_id,
sale_id::numeric,
sale_date::date,
sub_total::numeric,
tax::numeric,
total_amount::numeric
FROM stage_1_peak_sales
ORDER BY stage_id
"""

my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)

,stage_id,sale_id,sale_date,sub_total,tax,total_amount
0,1,5763728874,2020-10-03,12,0,12
1,2,5763729036,2020-10-03,72,0,72
2,3,5763728904,2020-10-03,24,0,24
3,4,5763728973,2020-10-03,96,0,96
4,5,5763728757,2020-10-03,108,0,108
...,...,...,...,...,...,...
92,93,5763728927,2020-10-03,72,0,72
93,94,5763729096,2020-10-03,48,0,48
94,95,5763729268,2020-10-03,84,0,84
95,96,5763729237,2020-10-03,60,0,60


# Validate the data types in the staging table stage_1_peak_stores

* sale_id - validate that it is numeric
* location_id - validate that it is numeric


In [6]:
rollback_before_flag = True
rollback_after_flag = True

query = """

SELECT
stage_id,
sale_id::numeric,
location_id::numeric
FROM
stage_1_peak_stores
ORDER BY stage_id

"""

my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)

,stage_id,sale_id,location_id
0,1,5763728874,12573
1,2,5763729036,12573
2,3,5763728904,12573
3,4,5763728973,12573
4,5,5763728757,12573
...,...,...,...
92,93,5763728927,12573
93,94,5763729096,12573
94,95,5763729268,12573
95,96,5763729237,12573


# Validate the data types in the staging table stage_1_peak_customers

* sale_id - validate that it is numeric
* customer_id - validate that it is numeric

In [7]:
rollback_before_flag = True
rollback_after_flag = True

query = """

SELECT
stage_id,
sale_id::numeric,
customer_id::numeric
FROM
stage_1_peak_customers
ORDER BY stage_id


"""

my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)

,stage_id,sale_id,customer_id
0,1,5763728874,3728404
1,2,5763729036,3729309
2,3,5763728904,3728508
3,4,5763728973,3728534
4,5,5763728757,3729188
...,...,...,...
92,93,5763728927,3728568
93,94,5763729096,3728990
94,95,5763729268,3728901
95,96,5763729237,3729019


# Validate the data types in the staging table stage_1_peak_line_items

* sale_id - validate that it is numeric
* line_item_id - validate that it is numeric
* product_id - validate that it is numeric
* price - validate that it is numeric
* quantity - validate that it is numeric

In [8]:
rollback_before_flag = True
rollback_after_flag = True

query = """

SELECT
stage_id,
sale_id::numeric,
line_item_id::numeric,
product_id::numeric,
price::numeric,
quantity::numeric
FROM stage_1_peak_line_items
ORDER BY stage_id

"""

my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)

,stage_id,sale_id,line_item_id,product_id,price,quantity
0,1,5763728874,1,42314680,12,1
1,2,5763729036,1,42314677,12,1
2,3,5763729036,2,42314682,12,3
3,4,5763729036,3,42314684,12,2
4,5,5763728904,1,42314680,12,1
...,...,...,...,...,...,...
347,348,5763729237,2,42314678,12,2
348,349,5763729237,3,42314682,12,2
349,350,5763728673,1,42314677,12,2
350,351,5763728673,2,42314678,12,1


# Validate the math on sub_total, tax, and total_amount in stage_1_peak_sales

total_amount = sub_total + tax


In [9]:
rollback_before_flag = True
rollback_after_flag = True

query = """

select stage_id
from stage_1_peak_sales
where total_amount::numeric <> (sub_total::numeric + tax::numeric)
ORDER BY stage_id
"""

my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)

,stage_id


# Validate the math between stage_1_peak_sales and stage_1_peak_line_items

total_sales in stage_1_peak_sales matches the sum of (price x quantity) in stage_1_peak_line_items


In [10]:
rollback_before_flag = True
rollback_after_flag = True

query = """

select a.stage_id
from stage_1_peak_sales as a
where total_amount::numeric <> (select sum(quantity::numeric*price::numeric) 
                                from stage_1_peak_line_items as b  
                                where a.sale_id = b.sale_id)
ORDER BY stage_id

"""

my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)

,stage_id


# Validate the tax is always zero in stage_1_peak_sales

In [11]:
rollback_before_flag = True
rollback_after_flag = True

query = """

select stage_id 
from stage_1_peak_sales
where tax::numeric <> 0
ORDER BY stage_id
"""

my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)

,stage_id


# Validate the price is always 12 in stage_1_peak_line_items

In [12]:
rollback_before_flag = True
rollback_after_flag = True

query = """

select stage_id
from stage_1_peak_line_items
where price::numeric <> 12
ORDER BY stage_id
"""

my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)

,stage_id


# Validate taxable is always N in stage_1_peak_line_items

In [13]:
rollback_before_flag = True
rollback_after_flag = True

query = """

select stage_id
from stage_1_peak_line_items
where taxable <> 'N'
order by stage_id


"""

my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)

,stage_id


# Validate the store is the same for all in stage_1_peak_stores

In [14]:
rollback_before_flag = True
rollback_after_flag = True

query = """

select ps.location_id, 
       ps.name,
       ps.street,
       ps.city,
       ps.state,
       ps.zip,
       count(*) as number_of_duplicates
from stage_1_peak_stores as ps
group by ps.location_id, ps.name, ps.street, ps.city, ps.state, ps.zip
having count(*) != (select count(*) from stage_1_peak_stores)

"""

my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)

,location_id,name,street,city,state,zip,number_of_duplicates


# Validate the product_id in stage_1_peak_line_items against peak_product_mapping

In [15]:
rollback_before_flag = True
rollback_after_flag = True

query = """

select stage_id
from stage_1_peak_line_items
where product_id::numeric in (select product_id::numeric from peak_product_mapping)
order by stage_id



"""

my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)

,stage_id
